## This part of code is in order to:
    1- Deal  with extracting information from raw files
    2- Create the source dataframe to work with.
    3- Create Voice_Sources.csv for use in next modules.

In [1]:
import pandas as pd
import re
import scipy.stats as stats
from scipy.io import wavfile
import numpy as np
import os

In [2]:
project_folder = './TEST_CODE/'

In [3]:
voices =  [voice for voice in os.listdir(project_folder) if os.path.isdir(os.path.join(project_folder, voice))]
size_voices = len(voices)

In [7]:
cols=['mean','std','median','first_q', 'third_q', 'inter_q', 'skewness', 'kurtosis','mode','peak', 'date','user', 'age', 'pronounce', 'sex']

df_source = pd.DataFrame(columns=cols)

In [8]:
df_source.head()

,mean,std,median,first_q,third_q,inter_q,skewness,kurtosis,mode,peak,date,user,age,pronounce,sex


In [9]:
# Here to make sure that right information is extracted, the pattern objects are defined using regular expressions.

# Dates are 8 digits with potential digits of 0 to 9, so to find the date we can search the patterns as :
date_pat = re.compile('[0-9]{8}')


# Investigating Readme files, shows that they are not following the same dictatation of lower/capital letters so:


# for age:
adult_pat = re.compile('[Aa]dult')
youth_pat = re.compile('[Yy]outh')
senior_pat = re.compile('[Ss]enior')

# for pronounciation:
american_pat = re.compile('[Aa]merican')
british_pat = re.compile('[Bb]ritish')
european_pat = re.compile('[Ee]uropean')
indian_pat = re.compile('[Ii]ndian')
australian_pat = re.compile('[Aa]ustralian')

#for sex:
female_pat = re.compile('[Ff]emale')
male_pat = re.compile('[Mm]ale')

In [10]:
def extract_voice_features(f):
    n_obs, minmax, mean, variance, skewness, kurtosis = stats.describe(f)
    median    = np.median(f)
    mode      = stats.mode(f).mode[0]
    std       = np.std(f)
    first_q ,third_q= np.percentile(f, [25 ,75])
    inter_q       = third_q - first_q
    low,peak  = minmax
    return mean,std ,median, first_q, third_q,inter_q, skewness, kurtosis, mode, peak

In [11]:
def extract_date(filename):

    try:
        date = date_pat.search(filename).group()
    except AttributeError:
        date = '20000000'
    return date

In [30]:
def extract_user_features(textfile):

    sex, age, pronounce,usr = 'unknonwn', 'unknonwn', 'unknonwn','unknonwn' 
    for line in open(textfile):
        if line.startswith("User Name:"): 
            usr = line.split(":")[1].strip()
        elif line.startswith("Age Range:"): 
            age = line.split(":")[1].strip()
        elif line.startswith("Pronunciation dialect:"): 
            pronounce = line.split(":")[1].strip()
        elif line.startswith("Gender:"): 
            sex = line.split(":")[1].strip()    
    return usr,age,pronounce,sex

In [31]:
def format_check(age, pronounce, sex):

    #age check
    if adult_pat.search(age)   : age = 'Adult' 
    elif youth_pat.search(age) : age = 'Youth'
    elif senior_pat.search(age): age = 'Senior'
    else: age = 'Unknown'
        
    #pronouncecheck:
    if british_pat.search(pronounce)     : pronounce = 'British' 
    elif american_pat.search(pronounce)  : pronounce = 'American'
    elif european_pat.search(pronounce)  : pronounce = 'European'
    elif indian_pat.search(pronounce)    : pronounce = 'Indian'
    elif australian_pat.search(pronounce): pronounce = 'Australian'
    else: pronounce = 'Unknown'

    #sex check
    if female_pat.search(sex): sex = 'F' 
    elif male_pat.search(sex): sex = 'M'
    else: sex = 'Unknown'
    
    return age,pronounce,sex

In [14]:
'''
    extract list of dominant frequencies in sliding windows of duration defined by 'step' for each of the 10 wav files and return an array
    frequencies_lol: list of lists
    every item in this list will contain 10 lists corresponding to each of the 10 wav files in every sample
    and the lists within the list will contain a range of *filtered* frequencies corresponding to sliding windows within each wav file
    '''
#get dominating frequencies in sliding windows of 100ms
#3200 sampling points every 1/10 sec 
#fftq tells you the frequencies associated with the coefficients

"\n    extract list of dominant frequencies in sliding windows of duration defined by 'step' for each of the 10 wav files and return an array\n    frequencies_lol: list of lists\n    every item in this list will contain 10 lists corresponding to each of the 10 wav files in every sample\n    and the lists within the list will contain a range of *filtered* frequencies corresponding to sliding windows within each wav file\n    "

In [37]:
def extract_freq(voice_folder):
    
    f_listoflists = [] 
    for wav in os.listdir(voice_folder):
        fs, data = wavfile.read(os.path.join(voice_folder, wav))

        windowsize = fs//10
        window_f = []
        for i in range(0,len(data),windowsize):
            fourier = np.fft.fft(data[i:i+windowsize])
            freqs = np.fft.fftfreq(len(fourier)) 
            imax = np.argmax(np.abs(fourier))
            frequency = freqs[imax]
            frequency_hz = abs(frequency *fs)
            window_f.append(frequency_hz)
            filtered_frequencies = [f for f in window_f if 0<f<280] # TO FILTER ONLY HUMAN VOICE RANGE
       
        f_listoflists.append(filtered_frequencies)

    freqs = [fr for sublist in f_listoflists for fr in sublist]

    return freqs

In [52]:
fs, data = wavfile.read('./test/_caustic_-20170306-smy/wav/en-0186.wav')

freq_from_fft(fs,data)





112.26530267826612

In [75]:
folder = './test/_caustic_-20170306-smy/wav'
extract_meanfund_freq(folder)

118.94245615428785

In [16]:
            #for some of the files (ex: Aaron-20130527-giy) 
            #I only recover frequencies of 0.0 (even if I don't split in chunks) which is not integrated into my lol and frequencies is empty
            #some of the samples don't contain a wav folder (Ex: 'LunaTick-20080329-vf1')

In [40]:
for i in range(size_voices):
        
    #Extract METADATA
    voice = sorted(voices)[i]
    folder = os.path.join(project_folder, voice)
    wav_folder = os.path.join(folder, 'wav')
    readme_file = os.path.join(folder, 'etc', 'README')

    date = extract_date(voice)
    
    #Check FORMAT of METADATA
    if os.path.isfile(readme_file):
            usr,age,pronounce,sex = extract_user_features(readme_file)
    age,pronounce,sex = format_check(age, pronounce, sex)
        
    #EXTRACT FREQUENCY FEATURES        
    if os.path.isdir(wav_folder): 
        frequencies = extract_freq(wav_folder)
        if len(frequencies) > 10: 

            mean,std ,median, first_q, third_q,inter_q, skewness, kurtosis, mode, peak = extract_voice_features(frequencies)
            my_dict = { 'mean':mean, 'std':std, 'median':median, 'first_q':first_q,
                           'third_q': third_q, 'inter_q':inter_q, 'skewness':skewness,
                           'kurtosis':kurtosis, 'mode': mode, 'peak':peak,'user':usr, 
                           'date':date, 'age':age, 'pronounce':pronounce,'sex':sex}
            print("\naddinng features of wave folder number %s out of %s total wave folders"%(i+1,size_voices))
            df_source.loc[i] = pd.Series(my_dict)
        
    df_source.to_csv('Voice_Sources.csv')


addinng features of wave folder number 1 out of 6245 total wave folders

addinng features of wave folder number 2 out of 6245 total wave folders

addinng features of wave folder number 3 out of 6245 total wave folders

addinng features of wave folder number 4 out of 6245 total wave folders

addinng features of wave folder number 5 out of 6245 total wave folders

addinng features of wave folder number 6 out of 6245 total wave folders

addinng features of wave folder number 7 out of 6245 total wave folders

addinng features of wave folder number 8 out of 6245 total wave folders

addinng features of wave folder number 9 out of 6245 total wave folders

addinng features of wave folder number 10 out of 6245 total wave folders

addinng features of wave folder number 11 out of 6245 total wave folders

addinng features of wave folder number 12 out of 6245 total wave folders

addinng features of wave folder number 13 out of 6245 total wave folders

addinng features of wave folder number 14 out 


addinng features of wave folder number 125 out of 6245 total wave folders

addinng features of wave folder number 126 out of 6245 total wave folders

addinng features of wave folder number 127 out of 6245 total wave folders

addinng features of wave folder number 128 out of 6245 total wave folders

addinng features of wave folder number 129 out of 6245 total wave folders

addinng features of wave folder number 130 out of 6245 total wave folders

addinng features of wave folder number 131 out of 6245 total wave folders

addinng features of wave folder number 132 out of 6245 total wave folders

addinng features of wave folder number 133 out of 6245 total wave folders

addinng features of wave folder number 138 out of 6245 total wave folders

addinng features of wave folder number 139 out of 6245 total wave folders

addinng features of wave folder number 140 out of 6245 total wave folders

addinng features of wave folder number 141 out of 6245 total wave folders

addinng features of wave


addinng features of wave folder number 241 out of 6245 total wave folders

addinng features of wave folder number 242 out of 6245 total wave folders

addinng features of wave folder number 243 out of 6245 total wave folders

addinng features of wave folder number 244 out of 6245 total wave folders

addinng features of wave folder number 245 out of 6245 total wave folders

addinng features of wave folder number 246 out of 6245 total wave folders

addinng features of wave folder number 247 out of 6245 total wave folders

addinng features of wave folder number 248 out of 6245 total wave folders

addinng features of wave folder number 249 out of 6245 total wave folders

addinng features of wave folder number 250 out of 6245 total wave folders

addinng features of wave folder number 251 out of 6245 total wave folders

addinng features of wave folder number 252 out of 6245 total wave folders

addinng features of wave folder number 253 out of 6245 total wave folders

addinng features of wave


addinng features of wave folder number 358 out of 6245 total wave folders

addinng features of wave folder number 359 out of 6245 total wave folders

addinng features of wave folder number 360 out of 6245 total wave folders

addinng features of wave folder number 361 out of 6245 total wave folders

addinng features of wave folder number 362 out of 6245 total wave folders

addinng features of wave folder number 363 out of 6245 total wave folders

addinng features of wave folder number 364 out of 6245 total wave folders

addinng features of wave folder number 365 out of 6245 total wave folders

addinng features of wave folder number 366 out of 6245 total wave folders

addinng features of wave folder number 367 out of 6245 total wave folders

addinng features of wave folder number 368 out of 6245 total wave folders

addinng features of wave folder number 369 out of 6245 total wave folders

addinng features of wave folder number 370 out of 6245 total wave folders

addinng features of wave


addinng features of wave folder number 505 out of 6245 total wave folders

addinng features of wave folder number 506 out of 6245 total wave folders

addinng features of wave folder number 507 out of 6245 total wave folders

addinng features of wave folder number 508 out of 6245 total wave folders

addinng features of wave folder number 509 out of 6245 total wave folders

addinng features of wave folder number 510 out of 6245 total wave folders

addinng features of wave folder number 511 out of 6245 total wave folders

addinng features of wave folder number 512 out of 6245 total wave folders

addinng features of wave folder number 513 out of 6245 total wave folders

addinng features of wave folder number 514 out of 6245 total wave folders

addinng features of wave folder number 515 out of 6245 total wave folders

addinng features of wave folder number 516 out of 6245 total wave folders

addinng features of wave folder number 517 out of 6245 total wave folders

addinng features of wave

...


addinng features of wave folder number 5666 out of 6245 total wave folders

addinng features of wave folder number 5667 out of 6245 total wave folders

addinng features of wave folder number 5668 out of 6245 total wave folders

addinng features of wave folder number 5669 out of 6245 total wave folders

addinng features of wave folder number 5670 out of 6245 total wave folders

addinng features of wave folder number 5671 out of 6245 total wave folders

addinng features of wave folder number 5672 out of 6245 total wave folders

addinng features of wave folder number 5673 out of 6245 total wave folders

addinng features of wave folder number 5674 out of 6245 total wave folders

addinng features of wave folder number 5675 out of 6245 total wave folders

addinng features of wave folder number 5676 out of 6245 total wave folders

addinng features of wave folder number 5677 out of 6245 total wave folders

addinng features of wave folder number 5678 out of 6245 total wave folders

addinng fea


addinng features of wave folder number 5789 out of 6245 total wave folders

addinng features of wave folder number 5790 out of 6245 total wave folders

addinng features of wave folder number 5791 out of 6245 total wave folders

addinng features of wave folder number 5792 out of 6245 total wave folders

addinng features of wave folder number 5793 out of 6245 total wave folders

addinng features of wave folder number 5794 out of 6245 total wave folders

addinng features of wave folder number 5795 out of 6245 total wave folders

addinng features of wave folder number 5796 out of 6245 total wave folders

addinng features of wave folder number 5804 out of 6245 total wave folders

addinng features of wave folder number 5806 out of 6245 total wave folders

addinng features of wave folder number 5807 out of 6245 total wave folders

addinng features of wave folder number 5808 out of 6245 total wave folders

addinng features of wave folder number 5809 out of 6245 total wave folders

addinng fea


addinng features of wave folder number 5926 out of 6245 total wave folders

addinng features of wave folder number 5927 out of 6245 total wave folders

addinng features of wave folder number 5928 out of 6245 total wave folders

addinng features of wave folder number 5929 out of 6245 total wave folders

addinng features of wave folder number 5930 out of 6245 total wave folders

addinng features of wave folder number 5931 out of 6245 total wave folders

addinng features of wave folder number 5932 out of 6245 total wave folders

addinng features of wave folder number 5933 out of 6245 total wave folders

addinng features of wave folder number 5934 out of 6245 total wave folders

addinng features of wave folder number 5935 out of 6245 total wave folders

addinng features of wave folder number 5936 out of 6245 total wave folders

addinng features of wave folder number 5937 out of 6245 total wave folders

addinng features of wave folder number 5938 out of 6245 total wave folders

addinng fea


addinng features of wave folder number 6034 out of 6245 total wave folders

addinng features of wave folder number 6035 out of 6245 total wave folders

addinng features of wave folder number 6036 out of 6245 total wave folders

addinng features of wave folder number 6037 out of 6245 total wave folders

addinng features of wave folder number 6038 out of 6245 total wave folders

addinng features of wave folder number 6039 out of 6245 total wave folders

addinng features of wave folder number 6040 out of 6245 total wave folders

addinng features of wave folder number 6041 out of 6245 total wave folders

addinng features of wave folder number 6042 out of 6245 total wave folders

addinng features of wave folder number 6043 out of 6245 total wave folders

addinng features of wave folder number 6044 out of 6245 total wave folders

addinng features of wave folder number 6045 out of 6245 total wave folders

addinng features of wave folder number 6046 out of 6245 total wave folders

addinng fea


addinng features of wave folder number 6145 out of 6245 total wave folders

addinng features of wave folder number 6146 out of 6245 total wave folders

addinng features of wave folder number 6147 out of 6245 total wave folders

addinng features of wave folder number 6148 out of 6245 total wave folders

addinng features of wave folder number 6149 out of 6245 total wave folders

addinng features of wave folder number 6150 out of 6245 total wave folders

addinng features of wave folder number 6151 out of 6245 total wave folders

addinng features of wave folder number 6152 out of 6245 total wave folders

addinng features of wave folder number 6153 out of 6245 total wave folders

addinng features of wave folder number 6154 out of 6245 total wave folders

addinng features of wave folder number 6155 out of 6245 total wave folders

addinng features of wave folder number 6156 out of 6245 total wave folders

addinng features of wave folder number 6157 out of 6245 total wave folders

addinng fea

In [41]:
df_source.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5537 entries, 0 to 6244
Data columns (total 15 columns):
mean         5537 non-null float64
std          5537 non-null float64
median       5537 non-null float64
first_q      5537 non-null float64
third_q      5537 non-null float64
inter_q      5537 non-null float64
skewness     5537 non-null float64
kurtosis     5537 non-null float64
mode         5537 non-null float64
peak         5537 non-null float64
date         5537 non-null object
user         5537 non-null object
age          5537 non-null object
pronounce    5537 non-null object
sex          5537 non-null object
dtypes: float64(10), object(5)
memory usage: 852.1+ KB


## DONE! Lets go for some more exploration using this initial dataframe in file 03_data_manipulation_01

## This part of code is in order to: apply a new range(80<freq<260) of frequencies to the filtering and redo the feature extraction!
    1- Deal  with extracting information from raw files
    2- Create the source dataframe to work with.
    3- Create Voice_Sources1.csv for use in next modules.

In [81]:
cols1=['mean','std','median','first_q', 'third_q', 'inter_q', 'skewness', 'kurtosis','mode','peak', 'date','user', 'age', 'pronounce', 'sex']

df_source1 = pd.DataFrame(columns=cols1)

In [82]:
def extract_freq_refilter(voice_folder):
    
    f_listoflists = [] 
    for wav in os.listdir(voice_folder):
        fs, data = wavfile.read(os.path.join(voice_folder, wav))

        windowsize = fs//10
        window_f = []
        for i in range(0,len(data),windowsize):
            fourier = np.fft.fft(data[i:i+windowsize])
            freqs = np.fft.fftfreq(len(fourier)) 
            imax = np.argmax(np.abs(fourier))
            frequency = freqs[imax]
            frequency_hz = abs(frequency *fs)
            window_f.append(frequency_hz)
            filtered_frequencies = [f for f in window_f if 80<f<260] # Refilter ON HUMAN VOICE RANGE
       
        f_listoflists.append(filtered_frequencies)

    freqs = [fr for sublist in f_listoflists for fr in sublist]

    return freqs

In [83]:
for i in range(size_voices):
        
    #Extract METADATA
    voice = sorted(voices)[i]
    folder = os.path.join(project_folder, voice)
    wav_folder = os.path.join(folder, 'wav')
    readme_file = os.path.join(folder, 'etc', 'README')

    date = extract_date(voice)
    
    #Check FORMAT of METADATA
    if os.path.isfile(readme_file):
            usr,age,pronounce,sex = extract_user_features(readme_file)
    age,pronounce,sex = format_check(age, pronounce, sex)
        
    #EXTRACT FREQUENCY FEATURES        
    if os.path.isdir(wav_folder): 
        frequencies = extract_freq_refilter(wav_folder)
        if len(frequencies) > 10: 

            mean,std ,median, first_q, third_q,inter_q, skewness, kurtosis, mode, peak = extract_voice_features(frequencies)
            my_dict = { 'mean':mean, 'std':std, 'median':median, 'first_q':first_q,
                           'third_q': third_q, 'inter_q':inter_q, 'skewness':skewness,
                           'kurtosis':kurtosis, 'mode': mode, 'peak':peak,'user':usr, 
                           'date':date, 'age':age, 'pronounce':pronounce,'sex':sex}
            print("\naddinng features of wave folder number %s out of %s total wave folders"%(i+1,size_voices))
            df_source1.loc[i] = pd.Series(my_dict)
        
    df_source1.to_csv('Voice_Sources1.csv')


addinng features of wave folder number 1 out of 6245 total wave folders

addinng features of wave folder number 2 out of 6245 total wave folders

addinng features of wave folder number 3 out of 6245 total wave folders

addinng features of wave folder number 4 out of 6245 total wave folders

addinng features of wave folder number 5 out of 6245 total wave folders

addinng features of wave folder number 6 out of 6245 total wave folders

addinng features of wave folder number 9 out of 6245 total wave folders

addinng features of wave folder number 10 out of 6245 total wave folders

addinng features of wave folder number 11 out of 6245 total wave folders

addinng features of wave folder number 12 out of 6245 total wave folders

addinng features of wave folder number 13 out of 6245 total wave folders

addinng features of wave folder number 14 out of 6245 total wave folders

addinng features of wave folder number 15 out of 6245 total wave folders

addinng features of wave folder number 16 ou


addinng features of wave folder number 130 out of 6245 total wave folders

addinng features of wave folder number 131 out of 6245 total wave folders

addinng features of wave folder number 132 out of 6245 total wave folders

addinng features of wave folder number 133 out of 6245 total wave folders

addinng features of wave folder number 138 out of 6245 total wave folders

addinng features of wave folder number 139 out of 6245 total wave folders

addinng features of wave folder number 140 out of 6245 total wave folders

addinng features of wave folder number 141 out of 6245 total wave folders

addinng features of wave folder number 142 out of 6245 total wave folders

addinng features of wave folder number 143 out of 6245 total wave folders

addinng features of wave folder number 144 out of 6245 total wave folders

addinng features of wave folder number 145 out of 6245 total wave folders

addinng features of wave folder number 146 out of 6245 total wave folders

addinng features of wave


addinng features of wave folder number 262 out of 6245 total wave folders

addinng features of wave folder number 264 out of 6245 total wave folders

addinng features of wave folder number 265 out of 6245 total wave folders

addinng features of wave folder number 266 out of 6245 total wave folders

addinng features of wave folder number 267 out of 6245 total wave folders

addinng features of wave folder number 268 out of 6245 total wave folders

addinng features of wave folder number 269 out of 6245 total wave folders

addinng features of wave folder number 270 out of 6245 total wave folders

addinng features of wave folder number 271 out of 6245 total wave folders

addinng features of wave folder number 272 out of 6245 total wave folders

addinng features of wave folder number 273 out of 6245 total wave folders

addinng features of wave folder number 274 out of 6245 total wave folders

addinng features of wave folder number 275 out of 6245 total wave folders

addinng features of wave


addinng features of wave folder number 404 out of 6245 total wave folders

addinng features of wave folder number 405 out of 6245 total wave folders

addinng features of wave folder number 406 out of 6245 total wave folders

addinng features of wave folder number 407 out of 6245 total wave folders

addinng features of wave folder number 408 out of 6245 total wave folders

addinng features of wave folder number 409 out of 6245 total wave folders

addinng features of wave folder number 410 out of 6245 total wave folders

addinng features of wave folder number 411 out of 6245 total wave folders

addinng features of wave folder number 412 out of 6245 total wave folders

addinng features of wave folder number 413 out of 6245 total wave folders

addinng features of wave folder number 414 out of 6245 total wave folders

addinng features of wave folder number 415 out of 6245 total wave folders

addinng features of wave folder number 416 out of 6245 total wave folders

addinng features of wave


addinng features of wave folder number 533 out of 6245 total wave folders

addinng features of wave folder number 534 out of 6245 total wave folders

addinng features of wave folder number 535 out of 6245 total wave folders

addinng features of wave folder number 537 out of 6245 total wave folders

addinng features of wave folder number 538 out of 6245 total wave folders

addinng features of wave folder number 539 out of 6245 total wave folders

addinng features of wave folder number 540 out of 6245 total wave folders

addinng features of wave folder number 541 out of 6245 total wave folders

addinng features of wave folder number 542 out of 6245 total wave folders

addinng features of wave folder number 543 out of 6245 total wave folders

addinng features of wave folder number 544 out of 6245 total wave folders

addinng features of wave folder number 545 out of 6245 total wave folders

addinng features of wave folder number 546 out of 6245 total wave folders

addinng features of wave

...


addinng features of wave folder number 5663 out of 6245 total wave folders

addinng features of wave folder number 5664 out of 6245 total wave folders

addinng features of wave folder number 5665 out of 6245 total wave folders

addinng features of wave folder number 5666 out of 6245 total wave folders

addinng features of wave folder number 5667 out of 6245 total wave folders

addinng features of wave folder number 5668 out of 6245 total wave folders

addinng features of wave folder number 5669 out of 6245 total wave folders

addinng features of wave folder number 5670 out of 6245 total wave folders

addinng features of wave folder number 5671 out of 6245 total wave folders

addinng features of wave folder number 5672 out of 6245 total wave folders

addinng features of wave folder number 5673 out of 6245 total wave folders

addinng features of wave folder number 5674 out of 6245 total wave folders

addinng features of wave folder number 5675 out of 6245 total wave folders

addinng fea


addinng features of wave folder number 5787 out of 6245 total wave folders

addinng features of wave folder number 5788 out of 6245 total wave folders

addinng features of wave folder number 5789 out of 6245 total wave folders

addinng features of wave folder number 5790 out of 6245 total wave folders

addinng features of wave folder number 5791 out of 6245 total wave folders

addinng features of wave folder number 5792 out of 6245 total wave folders

addinng features of wave folder number 5793 out of 6245 total wave folders

addinng features of wave folder number 5794 out of 6245 total wave folders

addinng features of wave folder number 5795 out of 6245 total wave folders

addinng features of wave folder number 5796 out of 6245 total wave folders

addinng features of wave folder number 5804 out of 6245 total wave folders

addinng features of wave folder number 5806 out of 6245 total wave folders

addinng features of wave folder number 5807 out of 6245 total wave folders

addinng fea


addinng features of wave folder number 5926 out of 6245 total wave folders

addinng features of wave folder number 5927 out of 6245 total wave folders

addinng features of wave folder number 5928 out of 6245 total wave folders

addinng features of wave folder number 5929 out of 6245 total wave folders

addinng features of wave folder number 5930 out of 6245 total wave folders

addinng features of wave folder number 5931 out of 6245 total wave folders

addinng features of wave folder number 5932 out of 6245 total wave folders

addinng features of wave folder number 5933 out of 6245 total wave folders

addinng features of wave folder number 5934 out of 6245 total wave folders

addinng features of wave folder number 5935 out of 6245 total wave folders

addinng features of wave folder number 5936 out of 6245 total wave folders

addinng features of wave folder number 5937 out of 6245 total wave folders

addinng features of wave folder number 5938 out of 6245 total wave folders

addinng fea


addinng features of wave folder number 6035 out of 6245 total wave folders

addinng features of wave folder number 6036 out of 6245 total wave folders

addinng features of wave folder number 6037 out of 6245 total wave folders

addinng features of wave folder number 6038 out of 6245 total wave folders

addinng features of wave folder number 6039 out of 6245 total wave folders

addinng features of wave folder number 6040 out of 6245 total wave folders

addinng features of wave folder number 6041 out of 6245 total wave folders

addinng features of wave folder number 6042 out of 6245 total wave folders

addinng features of wave folder number 6043 out of 6245 total wave folders

addinng features of wave folder number 6044 out of 6245 total wave folders

addinng features of wave folder number 6045 out of 6245 total wave folders

addinng features of wave folder number 6046 out of 6245 total wave folders

addinng features of wave folder number 6047 out of 6245 total wave folders

addinng fea


addinng features of wave folder number 6151 out of 6245 total wave folders

addinng features of wave folder number 6152 out of 6245 total wave folders

addinng features of wave folder number 6153 out of 6245 total wave folders

addinng features of wave folder number 6154 out of 6245 total wave folders

addinng features of wave folder number 6155 out of 6245 total wave folders

addinng features of wave folder number 6156 out of 6245 total wave folders

addinng features of wave folder number 6157 out of 6245 total wave folders

addinng features of wave folder number 6158 out of 6245 total wave folders

addinng features of wave folder number 6159 out of 6245 total wave folders

addinng features of wave folder number 6160 out of 6245 total wave folders

addinng features of wave folder number 6161 out of 6245 total wave folders

addinng features of wave folder number 6162 out of 6245 total wave folders

addinng features of wave folder number 6163 out of 6245 total wave folders

addinng fea

## DONE! Lets go for some more exploration using this initial dataframe in file 03_data_manipulation_02
